## Process _Stories_ dataset

This notebook prepares the _Stories_ dataset for the experiments in the Living Machines paper.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import re
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from pathlib import Path

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from tools import animacy_evaluation,processing

import spacy
import operator
nlp = spacy.load("en_core_web_lg")

### Parse corpus into sentence-level dataframe

In [ ]:
def parse_animacy_xml(animacy_file):
    tree = ET.parse(animacy_file)
    story = tree.getroot()
    fulltext = ""
    storyNumber = animacy_file.split("/")[-1].split(".sty")[0]
    
    dTokens = dict()
    dRefExp = dict()
    dCorExp = dict()
    
    dAnimacyExpressions = dict()
    processed_annotations = []
    
    # ------------------------------------------------------
    # Capture relevant data from xml: token, refering expression, and coreferent
    # expression annotated with animacy.
    for section in story:
        if section.attrib['id'] == 'edu.mit.parsing.token':
            for child in section.findall('./desc'):
                dTokens[child.attrib['id']] = child.text.strip()
                
        if section.attrib['id'] == 'edu.mit.discourse.rep.refexp':
            for child in section.findall('./desc'):
                reftokens = []
                offset = int(child.attrib['off'])
                grouped = child.text.split(",")
                for gr in grouped:
                    reftokens.append(gr.split("~"))
                dRefExp[child.attrib['id']] = (child.text, reftokens)
                        
        if section.attrib['id'] == 'edu.mit.discourse.rep.coref':
            for child in section.findall('./desc'):
                offset = int(child.attrib['off'])
                text2mask = child.text.split("|")[0]
                coref_ani = 0
                if 'ani' in child.attrib:
                    if child.attrib['ani'] == '1':
                        coref_ani = 1
                
                dCorExp[child.attrib['id']] = (child.text, coref_ani)
    
    # ------------------------------------------------------
    # Capture referring expressions whose coreferent is annotated with animacy
    for corefExp in dCorExp:
        coref_ani = dCorExp[corefExp][1]
        coreftext, refids = dCorExp[corefExp][0].split("|")
        refids = refids.split(",")
        for refid in refids:
            for sq in dRefExp[refid][1]:
                dAnimacyExpressions[tuple(sq)] = ([dTokens[tk] for tk in sq], coref_ani)
    
    # ------------------------------------------------------
    # Add sentence index to each token
    fulltext = [dTokens[t] for t in dTokens]
    
    tokens_in_sentence = dict()
    sIndex = 0
    sentences = sent_tokenize(" ".join(fulltext))
    sentences_indices = []
    dSentences = dict()
    
    for s in sentences:
        sIndex += 1
        sentences_indices.append((s, sIndex))
        dSentences[sIndex] = s
    
    current_sentence = sentences_indices[0][0].split(" ")
    for t in dTokens:
        if dTokens[t] == current_sentence[0]:
            tokens_in_sentence[t] = (dTokens[t], sentences_indices[0][1])
            del current_sentence[0]
            if len(current_sentence) == 0:
                del sentences_indices[0]
                if sentences_indices:
                    current_sentence = sentences_indices[0][0].split(" ")
        else:
            tokens_in_sentence[t] = (dTokens[t], None)
    
    dSentenceIndices = dict()
    for tk in tokens_in_sentence:
        word_index = tk
        sent_index = tokens_in_sentence[tk][1]
        if sent_index in dSentenceIndices:
            dSentenceIndices[sent_index].append(word_index)
        else:
            dSentenceIndices[sent_index] = [word_index]
        
    # ------------------------------------------------------
    # Recover sentence where animated expression occurs, and its context
    for expression in dAnimacyExpressions:
        sentences_involved = list(set([tokens_in_sentence[e][1] for e in expression]))
        sentence = 0
        if len(sentences_involved) > 1: # Discard multisentence expressions, often annotation errors it seems
            continue
        else:
            sentence = sentences_involved[0]
        
        token_expression = dAnimacyExpressions[expression][0]
        animacy = dAnimacyExpressions[expression][1]
        
        words_sentence = dSentences[sentence].split()
        index_sentence = dSentenceIndices[sentence]
        
        # ------------------------------------------------------
        # Find each item of the target expression in the current sentence, and mask it
        masked_sentence = []
        if len(words_sentence) == len(index_sentence):
            zipped_sentence = list(zip(dSentenceIndices[sentence], dSentences[sentence].split()))
            indexExp = 0
            for z in zipped_sentence:
                if indexExp < len(expression):
                    if z[0] == expression[indexExp] and z[1] == token_expression[indexExp]:
                        masked_sentence.append("[MASK]")
                        indexExp += 1
                    else:
                        masked_sentence.append(z[1])
                else:
                    masked_sentence.append(z[1])
        
        # ------------------------------------------------------
        # Relevant outputs:
        prev_sentence = dSentences.get(sentence - 1, "")
        current_sentence = dSentences[sentence]
        target_expression = " ".join(dAnimacyExpressions[expression][0]).strip()
        masked_sentence = " ".join(masked_sentence)
        next_sentence = dSentences.get(sentence + 1, "")
        # Replace a multi-token masked expression to just one mask, instead of
        # having consecutive masked elements. E.g. Instead of "[MASK] [MASK] [MASK]
        # drank" for "A little man drank", replace to "[MASK] drank":
        regex_mask = r"( ?\[MASK\] ?\-?)+" 
        masked_sentence = re.sub(regex_mask, " [MASK] ", masked_sentence).strip()
        
        # ------------------------------------------------------
        # Append relevant outputs to list to return:
        if masked_sentence.count("[MASK]") == 1: # This line is to filter out some inconsistencies in the data
            processed_annotations.append((storyNumber, prev_sentence, current_sentence, masked_sentence, next_sentence, target_expression, animacy))
    
    return processed_annotations

In [ ]:
# ------------------------------------------------------
# Process Jahan animacy corpus, turn it to Stories corpus:
storiesdf = pd.DataFrame()
for i in glob.glob("../resources/jahan_animacy_v1.0.0/jahan_animacy_corpus_v1.0.0/data/*"):
    processed_annotations = parse_animacy_xml(i)
    localdf = pd.DataFrame(processed_annotations, columns=["storyNumber", "prevSentence", "currentSentence", "maskedSentence", "nextSentence", "targetExpression", "animated"])
    storiesdf = pd.concat([storiesdf, localdf], ignore_index=True)
    
storiesdf[['maskedSentence', 'targetExpression']] = storiesdf.apply(lambda row: pd.Series(processing.processStories(row['targetExpression'], row['maskedSentence'], nlp)), axis=1)
storiesdf[['context3wmasked', 'context3w']] = storiesdf.apply(lambda row: pd.Series(processing.ngram_context(row['maskedSentence'], row['targetExpression'], 3)), axis=1)

In [ ]:
# ------------------------------------------------------
# Split stories into training and test set, and store:

trainsplit = 0.7

storiesdf = storiesdf.sample(frac=1, random_state=0).reset_index(drop=True)
train_set = storiesdf.sample(frac=trainsplit, random_state=0)
test_set = storiesdf[~storiesdf.index.isin(train_set.index)]

pathdf = "../data/stories/"
Path(pathdf).mkdir(parents=True, exist_ok=True)

storiesdf.to_pickle(pathdf + "all.pkl")
train_set.to_pickle(pathdf + "train.pkl")
test_set.to_pickle(pathdf + "test.pkl")

### Explore the data

In [ ]:
storiesdf.head()

In [ ]:
print("Train annd test sets")
print("====================")
print("Animacy train set:", train_set.animated.value_counts().to_dict())
print("Animacy test set:", test_set.animated.value_counts().to_dict())

# ------------------------------------------------------
# Annotation counts:

print("\nNumber of rows")
print("==============")
print("All rows:")
print("* All:", storiesdf.animated.count())
print("* Train:", train_set.animated.count())
print("* Test:", test_set.animated.count())

In [ ]:
# -----------------------------------
# Load a dataset:
df = pd.read_pickle("../data/stories/test.pkl")

sentInd = 21 # Select here one row in the dataframe
print("***prevSentence***", df.iloc[sentInd].prevSentence)
print("***currentSentence***", df.iloc[sentInd].currentSentence)
print("***maskedSentence***", df.iloc[sentInd].maskedSentence)
print("***nextSentence***", df.iloc[sentInd].nextSentence)
print("***targetExpression***", df.iloc[sentInd].targetExpression)
print("***context3wmasked***", df.iloc[sentInd].context3wmasked)
print("***context3w***", df.iloc[sentInd].context3w)
print("***animated***", df.iloc[sentInd].animated)

### Most frequent class baseline

In [ ]:
dataset_df = pd.read_pickle("../data/stories/test.pkl")
classes_by_frequency = dataset_df.animated.value_counts(normalize=True).to_dict()
most_frequent_class = max(classes_by_frequency.items(), key=operator.itemgetter(1))[0]

y_pred = [int(most_frequent_class) for x in dataset_df['animated'].tolist()]
y_true = [int(x) for x in dataset_df['animated'].tolist()]

precision, recall, fscore, micro_fscore, map_ = animacy_evaluation.results(y_true,y_pred,0.5)
print("p:", round(precision,3), "r:", round(recall,3), "macro_f1:", round(fscore,3), "micro_f1:", round(micro_fscore,3), "map:", round(map_,3))